# 情報科学演習II アルゴリズムとデータ構造II 第1週 2冊目

#### 第1週目次

1. ColabraoryのPythonノートブックに慣れる（1冊目）
2. グラフの最短経路問題を解く（2冊目）
3. グラフ操作ライブラリnetworkxに慣れる（2冊目）
4. リストと集合と辞書（3冊目）
5. 要素の探索にかかる速度を比べる（3冊目）
6. ダイクストラのアルゴリズムを理解する（4冊目）

## 2. グラフの最短経路問題を解く

次のようなグラフが与えられたとき，頂点0から頂点1への経路は赤線の例のように複数ある．
このような経路のうち距離が最も短い経路の長さを求めるのが最短経路問題である．
このグラフの頂点0から頂点1への最短経路は0-3-4-5-1で，この経路の長さは13である．

![](figs/graph-simple-0-1-paths.svg)

**練習**
グラフを書いた用紙を配布します．
グラフ上の頂点0からそれぞれの頂点1, ..., 7への最短経路を求め，用紙に用意した以下のような表に各頂点の最短<u>距離</u>を書きなさい．最短<u>経路</u>を書く必要はありません．

| 頂点 | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 |
| ---  | - | - | - | - | - | - | - | - |
| 距離 | 0 | *1 |   |   |   |   |   |   |

*1には頂点0から頂点1への最短距離を書く．他の頂点についても同様．

授業時間に指示したフォームに各頂点の距離を提出しなさい．

## 3. グラフ操作パッケージ networkx に慣れる

この章でグラフを扱う練習をします．
次回グラフの最短経路を求めるアルゴリズムをこのパッケージを使って作成するのでよく理解してください．

グラフ処理に[networkx](https://networkx.org)を使います．
グラフの可視化にgraphvizと[pygraphviz](https://pygraphviz.github.io)を使い，
画像の表示にIPython.display.[SVGクラス](https://ipython.readthedocs.io/en/stable/api/generated/IPython.display.html#IPython.display.SVG)とdisplay_svg関数を使います．

次の2つのセルでこれらのモジュールやパッケージを`import`します．
例えば，今後`nx.hoge()`と書いてあればnetworkxの`hoge`関数を呼び出しているということですし，
`SVG(ファイル名)`はファイルの内容を読み込んだオプジェクトを生成します．

一つ目のセルはパッケージを仮想マシンにインストールします．
`!apt`と`!pip`の2行で1分近く時間がかかります．
多くの行が表示されますが，最終行が次のようであれば，内容は気にしないで構いません．
行末のx.xxの部分は1.13などのバージョン番号です．
```
Successfully installed pygraphviz-x.xx
```

解説：[import文やfrom付きimport文](https://docs.python.org/ja/3/reference/simple_stmts.html#the-import-statement)

In [ ]:
!apt install libgraphviz-dev
!pip install pygraphviz

In [ ]:
import networkx as nx
import pygraphviz as pgv
import matplotlib.pyplot as plt
from IPython.display import SVG, display_svg

次のセルで無向グラフGを生成します．
空のグラフが生成されるので頂点はありません．
グラフの生成に使う関数はnetworkxパッケージのGraph関数です．

In [ ]:
G = nx.Graph()

次のセルでprint_graph関数とdraw_graph関数を定義しています．
これらの関数は今後変更したグラフを確認するために使います．
print_graph関数は引数で与えたグラフGの頂点のリストと辺のリストを出力する関数です．
draw_graph関数は引数で与えたグラフGを可視化して表示する関数です．

解説：
変数Aは[pygraphviz](https://pygraphviz.github.io)のグラフで，networkxのグラフGをpygraphvizで表示するためにAに変換しています．

解説：
頂点のリストはlist(G.nodes)で，辺のリストはlist(G.edges)で取得できます．
list(G.edges.data())は辺のリストで，辺の重みが付属するリストです．
辺の重みはあとで使います．

In [ ]:
def print_graph(G): # GはnetworkxのGraphクラス (networkx.classes.graph.Graphクラス)
  print('print_graph:')
  print('  nodes', list(G.nodes))
  if False: # Falseは辺の重みを出力しない．辺の重みを出力するならTrueにする．
    print('  edges', list(G.edges.data())) # 辺の重みも出力する
  else:
    print('  edges', list(G.edges)) # 辺の重みは出力しない

def draw_graph(G, filename = 'tmp'):
  A = nx.nx_agraph.to_agraph(G) # networkxのグラフGをpygraphviz用のグラフAに変換
  for edge in G.edges(data='weight'): # 辺のweightをlabelにコピー．drawで出力されるのはlabel．
    u, v, d = edge
    if d != None:
      A.get_edge(u, v).attr['label'] = d
  A.layout(args='-Nshape=circle')  # default prog='neato' 
  A.draw(F'{filename}.svg')
  display_svg(SVG(F'{filename}.svg'))

空のグラフ`G`に対してこれらの関数を呼び出してみましょう．

In [ ]:
print_graph(G)
draw_graph(G)

次のセルに示すように`add_node`関数はグラフ`G`に頂点を追加します．
辺の追加には`add_edge`関数を使います．
次のセルではグラフ`G`に二つの頂点0と1を追加し，辺(0,1)を追加しています．
無向グラフなので辺(0,1)と辺(1,0)は同じ一つの辺です．

In [ ]:
G.add_node(0)
G.add_node(1)
G.add_edge(0, 1)
print_graph(G)
draw_graph(G)

**練習**
グラフ`G`からすべての頂点と辺を削除するように，次のコードの`....`の部分に書きなさい．
グラフ`G`のすべての頂点を削除する関数は`clear()`です．
`G.clear()`のように呼び出します．

In [ ]:
....
print_graph(G)
draw_graph(G)

グラフ`G`に辺(0,1)を追加すると頂点0と1は自動的に追加されます．

In [ ]:
G.add_edge(0, 1)
print_graph(G)
draw_graph(G)

**練習**
2章で配布した用紙に書いたグラフと同じ頂点と枝をグラフ`G`に加えるコードを次のセルに書きなさい．
辺の重みについてはこの練習では無視してよい．

In [ ]:
...
print_graph(G)
draw_graph(G)

辺(0,1)に長さ（重み）4を追加するには次のように書きます．

In [ ]:
G[0][1]['weight'] = 4

辺のリストは`list(G.edges())`で得られますが，長さが含まれません．
長さを含めた辺のリストは`list(G.edges.data())`で得られます．

**練習**
次の2つのセルを実行して`.data`の有無で出力が異なることを確かめなさい．

In [ ]:
list(G.edges())

In [ ]:
list(G.edges.data())

**練習**
上で定義した`print_graph`関数を呼び出しても辺の重みが出力されません．
`print_graph`関数を書き換えて辺の重みが出力されるようにしなさい．
次のセルを実行して確かめるといいでしょう．

In [ ]:
print_graph(G)

**練習**
2章で配布した用紙に書いたグラフと同じ辺の長さをもつグラフ`G`になるように，辺に長さを追加するコードを次のセルに書きなさい．
辺の追加が正しくできたか`print_graph`関数と`draw_graph`関数を呼び出して確かめなさい．

このノートブックは以上です．次のノートブックに進みなさい．